In [1]:
import os

os.chdir('/home/enis/projects/nna/src/nna/exp/megan/run-1/')


In [2]:
# import run
# import nna
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import torchaudio
torchaudio.set_audio_backend("sox_io")

import numpy as np

/scratch/enis/conda/envs/soundenv3/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [3]:
import runconfigs
import wandb
from ignite.metrics import Accuracy, Loss
from ignite.contrib.metrics import ROC_AUC

In [4]:
# from nna.exp import augmentations,
from nna.exp import modelArchs,runutils

In [5]:
# wandb.init(config=runconfigs.default_config, project=runconfigs.PROJECT_NAME)
# config = wandb.config
config = runconfigs.default_config
# wandb.config.update(args) # adds all of the arguments as config variables

params = {
    'batch_size': config['batch_size'],
    'shuffle': True,
    'num_workers': 0
}


In [6]:
device = torch.device(
    f"cuda:{config['device']}" if torch.cuda.is_available() else "cpu")


In [7]:
from pathlib import Path
import numpy as np
from sklearn.model_selection import train_test_split

# labelsbyhumanpath = Path('/scratch/enis/data/nna/labeling/results/')
# sourcePath = Path("/scratch/enis/data/nna/labeling/splits/")


In [8]:
CATEGORY_COUNT = 2


In [9]:
from collections import Counter


In [10]:
 # RAW DATA
def load_raw_data(sample_count,CATEGORY_COUNT):
    sample_count = 1200
    X = np.empty((sample_count,480000),dtype=np.float32)
    y_true = np.random.randint(0,CATEGORY_COUNT,(sample_count))
    for i,y in enumerate(y_true):
        X[i,:] = y
    X = np.interp(X, (X.min(), X.max()), (-32768 , 32767))
    n_values = np.max(y_true) + 1
    y_true = np.eye(n_values)[y_true]
    X_train, X_test, y_train, y_test = train_test_split(
                    X, y_true, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(
                    X_train, y_train, test_size=0.25,random_state=42)
    return X_train,X_test,X_val,y_train,y_test,y_val

def mock_raw_data(sample_count,CATEGORY_COUNT):
#     sample_count = 30
    X = np.empty((sample_count*CATEGORY_COUNT,480000),dtype=np.float32)
    # y_true = np.random.randint(0,CATEGORY_COUNT,(sample_count))
    y_true=[]
    for i in range(CATEGORY_COUNT):
        y_true.extend([i]*sample_count)
    print(len(y_true))
    for i,y in enumerate(y_true):
        X[i,:] = y
    X = np.interp(X, (X.min(), X.max()), (0 , 3400))
#     X = np.interp(X, (X.min(), X.max()), (-32768 , 32767))
    n_values = np.max(y_true) + 1
    y_true = np.eye(n_values)[y_true]
    X_train, X_test, y_train, y_test = train_test_split(
                    X, y_true, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(
                    X_train, y_train, test_size=0.25,random_state=42)
#     print(y_train)
    return X_train,X_test,X_val,y_train,y_test,y_val
    


In [11]:
# X_train,X_test,X_val,y_train,y_test,y_val = load_raw_data('labelsbyhumanpath','sourcePath')
X_train,X_test,X_val,y_train,y_test,y_val = mock_raw_data(4,CATEGORY_COUNT)
X_train,X_test,X_val=torch.from_numpy(X_train).float(),torch.from_numpy(X_test).float(),torch.from_numpy(X_val).float()
y_train,y_test,y_val=torch.from_numpy(y_train).float(),torch.from_numpy(y_test).float(),torch.from_numpy(y_val).float()

# labelsbyhumanpath = Path('/scratch/enis/data/nna/labeling/results/')
# with open(labelsbyhumanpath/"np_array_Ymatrix.npy", 'rb') as f:
#     y_true = np.load(f)

Counter(np.argmax(y_train,axis=1).tolist()),Counter(np.argmax(y_val,axis=1).tolist()),Counter(np.argmax(y_test,axis=1).tolist())


8


(Counter({1: 2, 0: 2}), Counter({0: 1, 1: 1}), Counter({0: 1, 1: 1}))

In [12]:

# X_train2=np.interp(X_train, (X_train.min(), X_train.max()), (-32768 , 32767))
# torch.from_numpy(X_train2).float()


In [13]:
# y_val

In [14]:
# smaple_index=10
# X_train[smaple_index,:],y_train[smaple_index,]


In [15]:
# X_train[0,:].shape

In [16]:
# multiply(torch.ones((1,2)),torch.ones((1,2)))

In [17]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __init__(self, maxMelLen, sampling_rate):
        # sr = 44100 etc
        self.maxMelLen = maxMelLen
        self.sampling_rate = sampling_rate

    def __call__(self, sample):
        x, y = sample
        #https://github.com/PCerles/audio/blob/3803d0b27a4e13efa760227ef6c71d0f3753aa98/test/test_transforms.py#L262
        #librosa defaults
        n_fft = 2048
        hop_length = 512
        power = 2.0
        n_mels = 128
        n_mfcc = 40
        # htk is false in librosa, no setting in torchaudio -?
        # norm is 1 in librosa, no setting in torchaudio -?
        melspect_transform = torchaudio.transforms.MelSpectrogram(sample_rate=self.sampling_rate, window_fn=torch.hann_window,
                                                                  hop_length=hop_length, n_mels=n_mels, n_fft=n_fft)

    
        db_transform = torchaudio.transforms.AmplitudeToDB("power", 80.)
        mel = melspect_transform(x.reshape(-1))
        an_x = db_transform(mel)
        #librosa version
#         mel = librosa.feature.melspectrogram(y=x.reshape(-1),
#                                              sr=self.sampling_rate)
#         an_x = librosa.power_to_db(mel, ref=np.max)
#         an_x = an_x.astype("float32")
#         y = y.astype('float32')
        an_x = an_x[:, :self.maxMelLen]
        # 2-d conv
#         x = an_x.reshape(1, *an_x.shape[:])
        # 1-d conv
        x = an_x.reshape(1, an_x.shape[0]*an_x.shape[1])

        
        return x,y

# #test
# maxMelLen_test = 850
# SAMPLING_RATE_test = 48000
# sample_len_seconds = 10
# # to_tensor works on single sample
# sample_count = 1
# xx_test = torch.ones((sample_count,SAMPLING_RATE_test*sample_len_seconds))
# y_values = torch.ones(sample_count)
# 
# toTensor = ToTensor(maxMelLen_test,SAMPLING_RATE_test)
# x_out,y_out=toTensor((xx_test,y_values))
# x_out.shape,y_out.shape

In [18]:
# X_train,X_test,X_val,y_train,y_test,y_val
X_train[:].shape

torch.Size([4, 480000])

In [19]:
# toTensor = ToTensor(maxMelLen_test,SAMPLING_RATE_test)
# x_out2,y_out=toTensor((X_train[1:2,:],y_train))
# x_out.shape,y_out.shape

In [20]:
# torch.mean(x_out[0]),torch.mean(x_out2[0])
# y_train

In [21]:
# X_train,y_train

In [22]:

# pitch = augmentations.pitch_shift_n_stepsClass(
#     runconfigs.SAMPLING_RATE, config['pitch_shift_n_steps'])
# noise = augmentations.addNoiseClass(config['noise_factor'])
# strech = augmentations.time_stretchClass(runconfigs.SAMPLING_RATE*runconfigs.EXCERPT_LENGTH,
#                                             config['time_stretch_factor'],
#                                             isRandom=True)
# shift = augmentations.shiftClass(config['roll_rate'], isRandom=True)
maxMelLen = 850 # old 850
# toTensor = augmentations.ToTensor(maxMelLen,runconfigs.SAMPLING_RATE)
toTensor = ToTensor(maxMelLen,runconfigs.SAMPLING_RATE)

In [23]:
from ignite.metrics import EpochMetric


def roc_auc_perClass_compute_fn(y_preds, y_targets):
    try:
        from sklearn.metrics import roc_auc_score
    except ImportError:
        raise RuntimeError(
            "This contrib module requires sklearn to be installed.")

    y_true = y_targets.numpy()
    y_pred = y_preds.numpy()
#     print(y_pred,y_true)
#     res = []
#     for y_true_perClass_Index in y_true.shape[1]:
#         res.append(
#             roc_auc_score(y_true[:, y_true_perClass_Index],
#                           y_pred[:, y_true_perClass_Index]))
    res = roc_auc_score(y_true, y_pred, average=None)
    return res


#[docs]
class ROC_AUC_perClass(EpochMetric):
    """Computes Area Under the Receiver Operating Characteristic Curve (ROC AUC)
  accumulating predictions and the ground-truth during an epoch and applying
  `sklearn.metrics.roc_auc_score <http://scikit-learn.org/stable/modules/generated/
  sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score>`_ .

  Args:
      output_transform (callable, optional): a callable that is used to transform the
          :class:`~ignite.engine.engine.Engine`'s ``process_function``'s output into the
          form expected by the metric. This can be useful if, for example, you have a multi-output model and
          you want to compute the metric with respect to one of the outputs.
      check_compute_fn (bool): Optional default False. If True, `roc_curve
          <http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#
          sklearn.metrics.roc_auc_score>`_ is run on the first batch of data to ensure there are
          no issues. User will be warned in case there are any issues computing the function.

  ROC_AUC expects y to be comprised of 0's and 1's. y_pred must either be probability estimates or confidence
  values. To apply an activation to y_pred, use output_transform as shown below:

  .. code-block:: python

      def activated_output_transform(output):
          y_pred, y = output
          y_pred = torch.sigmoid(y_pred)
          return y_pred, y

      roc_auc = ROC_AUC(activated_output_transform)

  """
    def __init__(self,
                 output_transform=lambda x: x,
                 check_compute_fn: bool = False):
#         print(output_transform)
        super(ROC_AUC_perClass,
              self).__init__(roc_auc_perClass_compute_fn,
                             output_transform=output_transform,
                             check_compute_fn=check_compute_fn)


In [24]:

def conv_output_shape(h_w, kernel_size=1, stride=1, pad=0, dilation=1):
    """
  Utility function for computing output of convolutions
  takes a tuple of (h,w) and returns a tuple of (h,w)
  """
    from math import floor
    if type(kernel_size) is not tuple:
        kernel_size = (kernel_size, kernel_size)
    h = floor(((h_w[0] + (2 * pad) - (dilation *
                                      (kernel_size[0] - 1)) - 1) / stride) + 1)
    w = floor(((h_w[1] + (2 * pad) - (dilation *
                                      (kernel_size[1] - 1)) - 1) / stride) + 1)
    return h, w

# mel.shape,an_x.shape,X_train.shape
class testModel(nn.Module):
    '''A simple model for testing by overfitting.
    '''
    def __init__(self, out_channels, h_w, kernel_size, FLAT=False,output_shape=(10,)):
        # h_w: height will be always one since we use 1d convolution 
        super(testModel, self).__init__()
        self.out_channels = out_channels
        #### CONV
        self.conv1 = nn.Conv1d(in_channels=1, # depth of image == depth of filters
                               out_channels=self.out_channels, # number of filters 
                               kernel_size=kernel_size, # size of the filters/kernels
                               padding=1)

        self.conv1_shape = conv_output_shape(h_w, kernel_size=kernel_size, stride=1, pad=1, dilation=1)
        # conv is 1d
        self.conv1_shape = (1,self.conv1_shape[1])
        
        self.fc1 = nn.Linear(self.out_channels * self.conv1_shape[0] *self.conv1_shape[1], 100)  # 100

        self.fc2 = nn.Linear(100,output_shape[0])
        
    def forward(self, x):
#         x = x.reshape(1,)
#         print(x.shape) #  50,1,108800 (850*128)
        x = F.relu(self.conv1(x))
#         x = self.pool(x)
        # x = self.drop(x)
#         print(x.shape)# 58, 2, 108801
#         print(self.conv1_shape)
#         print(x.shape)
        x = x.view(-1, self.out_channels * self.conv1_shape[0] *self.conv1_shape[1])
        # batch_norm is missing
        x = F.relu((self.fc1(x)))
        x = (self.fc2(x))

#         x = self.drop(x)

#         x = self.fc4(x)
#         x = torch.sigmoid(x)
#                 x = F.log_softmax(x,dim=1)
        return x

# test
input_shape=(1,(850*128))
output_shape=(10,)
testModel_ins=testModel(out_channels=2,h_w=input_shape,kernel_size=2,output_shape=output_shape)
# a.conv1.weight
a_out=testModel_ins(torch.ones((3,1,input_shape[1])))

a_out_correct=torch.zeros(a_out.shape)
a_out_correct[0][:]=1
a_out_correct
a_out.detach().numpy()

torch.exp(a_out),a_out

(tensor([[1.1011, 0.9161, 0.8767, 1.0228, 1.0041, 0.9782, 1.1169, 1.0258, 1.0322,
          1.1230],
         [1.1011, 0.9161, 0.8767, 1.0228, 1.0041, 0.9782, 1.1169, 1.0258, 1.0322,
          1.1230],
         [1.1011, 0.9161, 0.8767, 1.0228, 1.0041, 0.9782, 1.1169, 1.0258, 1.0322,
          1.1230]], grad_fn=<ExpBackward>),
 tensor([[ 0.0963, -0.0876, -0.1316,  0.0226,  0.0041, -0.0220,  0.1106,  0.0254,
           0.0317,  0.1160],
         [ 0.0963, -0.0876, -0.1316,  0.0226,  0.0041, -0.0220,  0.1106,  0.0254,
           0.0317,  0.1160],
         [ 0.0963, -0.0876, -0.1316,  0.0226,  0.0041, -0.0220,  0.1106,  0.0254,
           0.0317,  0.1160]], grad_fn=<AddmmBackward>))

In [25]:

# for y_true_perClass_Index in a_out_correct.shape[1]:
#     print(y_true_perClass_Index)

In [26]:
# from ignite.contrib.metrics import ROC_AUC
# from nna.exp.metrics import ROC_AUC_perClass
def activated_output_transform(output):
    y_pred, y = output
#     y_pred = torch.exp(y_pred)
    return y_pred, y

In [27]:
asd=ROC_AUC_perClass(activated_output_transform)
asd.update((a_out,a_out_correct))
asd.compute()

array([0.25, 0.25, 0.25, 0.75, 0.5 , 0.5 , 0.5 , 0.75, 0.5 , 0.25])

In [28]:

transformCompose = transforms.Compose([
#     pitch,
#     strech,
#     shift,
#     noise,
    toTensor,
])


sound_datasets = {
    phase: runutils.audioDataset(XY[0], XY[1], transform=transformCompose)
    for phase, XY in
    zip(['train', 'val', 'test'],
        [[X_train, y_train], [X_val, y_val], [X_test, y_test]])
}

dataloaders = {
    x: torch.utils.data.DataLoader(sound_datasets[x], **params)
    for x in ['train', 'val', 'test']
}

# this will change
h_w = [128, 850]
kernel_size = (3, 3)
# if config['CNNLayer_count'] == 1:
#     model = modelArchs.NetCNN1(config['CNN_filters_1'], h_w,
#                                 kernel_size).float().to(device)

# if config['CNNLayer_count'] == 2:
#     model = modelArchs.NetCNN2(config['CNN_filters_1'], config.CNN_filters_2,
#                                 h_w, kernel_size,
#                                 kernel_size).float().to(device)

#simpler model

output_shape=(CATEGORY_COUNT,)
model = testModel(out_channels=2,h_w=(1,h_w[0]*h_w[1]),kernel_size=kernel_size[0]*kernel_size[0],output_shape=output_shape)
model.float().to(device)

# device is defined before

model.float().to(device)  # Move model before creating optimizer
optimizer = torch.optim.AdamW(model.parameters(),
#                                 weight_decay=config['weight_decay'],
                             )



criterion = nn.BCEWithLogitsLoss()
# statHistory={'valLoss':[],'trainLoss':[],'trainAUC':[],'valAUC':[]}

metrics = {
    'loss': Loss(criterion),  # 'accuracy': Accuracy(),
#     'ROC_AUC': ROC_AUC(runutils.activated_output_transform),
    'ROC_AUC': ROC_AUC_perClass(activated_output_transform),

}


In [29]:
model

testModel(
  (conv1): Conv1d(1, 2, kernel_size=(9,), stride=(1,), padding=(1,))
  (fc1): Linear(in_features=217588, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=2, bias=True)
)

In [30]:
trainloader = dataloaders['train']

In [31]:
# for epoch in range(20000):  # loop over the dataset multiple times

#     running_loss = 0.0
#     for i, data in enumerate(trainloader, 0):
#         # get the inputs; data is a list of [inputs, labels]
#         inputs, labels = data

#         # zero the parameter gradients
#         optimizer.zero_grad()
#         inputs=inputs.to(device)
#         # forward + backward + optimize
#         outputs = model(inputs)
#         outputs = outputs.to('cpu')
#         loss = criterion(outputs, labels)
# #         print(outputs)
# #         print(labels)
# #         print(loss)
#         loss.backward()
#         optimizer.step()

#         # print statistics
#         running_loss += loss.item()
# #         print(running_loss)
#         if epoch % 20 == 19:    # print every 2000 mini-batches
#             print('[%d, %5d] loss: %.3f' %
#                   (epoch + 1, i + 1, running_loss))
#             running_loss = 0.0

# print('Finished Training')

In [32]:
model.conv1.weight

Parameter containing:
tensor([[[-0.2701, -0.0257,  0.2114,  0.2575, -0.2305,  0.2908,  0.1457,
          -0.1355,  0.3114]],

        [[-0.2209, -0.0420, -0.0443,  0.2842, -0.3163,  0.0209,  0.2443,
          -0.2832,  0.2207]]], device='cuda:1', requires_grad=True)

In [33]:
model.conv1

Conv1d(1, 2, kernel_size=(9,), stride=(1,), padding=(1,))

In [34]:
model.fc1.weight

Parameter containing:
tensor([[ 0.0007, -0.0018,  0.0021,  ...,  0.0013, -0.0018, -0.0004],
        [ 0.0001,  0.0019,  0.0017,  ...,  0.0010,  0.0012, -0.0005],
        [ 0.0020, -0.0014, -0.0015,  ...,  0.0016, -0.0013,  0.0020],
        ...,
        [ 0.0016, -0.0013, -0.0010,  ..., -0.0014, -0.0006,  0.0020],
        [-0.0020,  0.0021, -0.0006,  ..., -0.0020,  0.0004, -0.0019],
        [-0.0001,  0.0007,  0.0020,  ...,  0.0001,  0.0009, -0.0013]],
       device='cuda:1', requires_grad=True)

In [35]:

print('ready ?')
runutils.run(model, dataloaders, optimizer, criterion, metrics, device,config, runconfigs.PROJECT_NAME)


Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: enisberk (use `wandb login --relogin` to force relogin)


ready ?


2021-01-08 05:00:28,314 Trainer INFO: Engine run starting with max_epochs=2000.
/scratch/enis/conda/envs/soundenv3/lib/python3.7/site-packages/torch/functional.py:516: UserWarning: stft will require the return_complex parameter be explicitly  specified in a future PyTorch release. Use return_complex=False  to preserve the current behavior or return_complex=True to return  a complex output. (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370141920/work/aten/src/ATen/native/SpectralOps.cpp:653.)
  normalized, onesided, return_complex)
/scratch/enis/conda/envs/soundenv3/lib/python3.7/site-packages/torch/functional.py:516: UserWarning: The function torch.rfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.fft or torch.fft.rfft. (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370141920/work/aten/src/ATen/native/SpectralOps.cpp:590.)
  normalized, onesided, retu

train loss 0.0
val loss 0.0
validation roc auc [1. 1.] 1


2021-01-08 05:00:31,599 Train Evaluator INFO: Epoch[1] Complete. Time taken: 00:00:00
2021-01-08 05:00:31,601 Train Evaluator INFO: Engine run complete. Time taken: 00:00:00
2021-01-08 05:00:31,602 Val Evaluator INFO: Engine run starting with max_epochs=1.
2021-01-08 05:00:31,646 Val Evaluator INFO: Epoch[1] Complete. Time taken: 00:00:00
2021-01-08 05:00:33,789 Val Evaluator INFO: Engine run complete. Time taken: 00:00:02
2021-01-08 05:00:33,796 Trainer INFO: Epoch[2] Complete. Time taken: 00:00:02
2021-01-08 05:00:33,904 Train Evaluator INFO: Engine run starting with max_epochs=1.
2021-01-08 05:00:33,959 Train Evaluator INFO: Epoch[1] Complete. Time taken: 00:00:00
2021-01-08 05:00:33,960 Train Evaluator INFO: Engine run complete. Time taken: 00:00:00
2021-01-08 05:00:33,962 Val Evaluator INFO: Engine run starting with max_epochs=1.


...

2021-01-08 05:04:56,031 Train Evaluator INFO: Engine run starting with max_epochs=1.
2021-01-08 05:04:56,114 Train Evaluator INFO: Epoch[1] Complete. Time taken: 00:00:00
2021-01-08 05:04:56,115 Train Evaluator INFO: Engine run complete. Time taken: 00:00:00
2021-01-08 05:04:56,116 Val Evaluator INFO: Engine run starting with max_epochs=1.
2021-01-08 05:04:56,164 Val Evaluator INFO: Epoch[1] Complete. Time taken: 00:00:00
2021-01-08 05:04:56,166 Val Evaluator INFO: Engine run complete. Time taken: 00:00:00
2021-01-08 05:04:56,171 Trainer INFO: Epoch[1332] Complete. Time taken: 00:00:00
2021-01-08 05:04:56,233 Train Evaluator INFO: Engine run starting with max_epochs=1.


train loss 0.0
val loss 0.0
validation roc auc [1. 1.] 1332


2021-01-08 05:04:56,448 Train Evaluator INFO: Epoch[1] Complete. Time taken: 00:00:00
2021-01-08 05:04:56,450 Train Evaluator INFO: Engine run complete. Time taken: 00:00:00
2021-01-08 05:04:56,451 Val Evaluator INFO: Engine run starting with max_epochs=1.
2021-01-08 05:04:56,491 Val Evaluator INFO: Epoch[1] Complete. Time taken: 00:00:00
2021-01-08 05:04:56,493 Val Evaluator INFO: Engine run complete. Time taken: 00:00:00
2021-01-08 05:04:56,498 Trainer INFO: Epoch[1333] Complete. Time taken: 00:00:00
2021-01-08 05:04:56,651 Trainer ERROR: Engine run is terminating due to exception: .


train loss 0.0
val loss 0.0
validation roc auc [1. 1.] 1333


KeyboardInterrupt: 

In [ ]:
model.fc1.weight

In [ ]:

example_input = (torch.ones((1,1,h_w[0]*h_w[1]))*-32767)
example_input = example_input.float().to(device)
out=model(example_input)
out

activated_output_transform((out,out))

In [ ]:
torch.Tensor([10,10,10])

In [ ]:
sum(model.fc1.weight)

In [ ]:
m = nn.Sigmoid()

loss = nn.BCEWithLogitsLoss()
input = torch.tensor([100.0,100,100], requires_grad=True)
# target = torch.empty(3).random_(2)
target = torch.ones(3)
output = loss((input), target)
input,target,output

In [ ]:
input

In [ ]:
target

In [ ]:
output

In [ ]:
m(input)